In [7]:
import tensorflow as tf

# shakespeare_url = "https://github.com/karpathy/char-rnn/blob/master/data/tinyshakespeare/input.txt"
# filepath = tf.keras.utils.get_file("shakespeare.txt",shakespeare_url)
with open("shakespeare.txt") as f:
    shakespeare_text = f.read()

In [8]:
print(shakespeare_text[:80])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.


In [9]:
text_vec_layer = tf.keras.layers.TextVectorization(split="character", standardize="lower")
text_vec_layer.adapt([shakespeare_text])
encoded = text_vec_layer([shakespeare_text])[0]

In [13]:
encoded -= 2
n_tokens = text_vec_layer.vocabulary_size() -2 
dataset_size = len(encoded)

In [14]:
dataset_size

1115393

In [15]:
def to_dataset(sequence, length, shuffle=False, seed=None, batch_size=32):
    ds = tf.data.Dataset.from_tensor_slices(sequence)
    ds = ds.window(length + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda window_ds: window_ds.batch(length + 1))
    if shuffle:
        ds = ds.shuffle(buffer_size=100_000, seed = seed)
    ds = ds.batch(batch_size)
    return ds.map(lambda window: (window[:, :-1], window[:, 1:])).prefetch(1)

In [16]:
length = 100
tf.random.set_seed(42)
train_set = to_dataset(encoded[:1_000_000],length=length, shuffle=True, seed=42)
valid_set = to_dataset(encoded[1_000_000:1_060_000],length=length)
test_set = to_dataset(encoded[1_060_000:],length=length)

In [17]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=n_tokens,output_dim=16),
    tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.Dense(n_tokens, activation="softmax")
])

model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])

model_ckpt = tf.keras.callbacks.ModelCheckpoint("my_shakespeare_model", monitor="val_accuracy", save_best_only=True)

history = model.fit(train_set,validation_data=valid_set,epochs=10, callbacks=[model_ckpt])

Epoch 1/10
  31247/Unknown - 860s 27ms/step - loss: 1.4062 - accuracy: 0.5700

INFO:tensorflow:Assets written to: my_shakespeare_model\assets


INFO:tensorflow:Assets written to: my_shakespeare_model\assets


31247/31247 [==============================] - 879s 28ms/step - loss: 1.4062 - accuracy: 0.5700 - val_loss: 1.6114 - val_accuracy: 0.5315
Epoch 2/10
31247/31247 [==============================] - ETA: 0s - loss: 1.3105 - accuracy: 0.5919

INFO:tensorflow:Assets written to: my_shakespeare_model\assets


INFO:tensorflow:Assets written to: my_shakespeare_model\assets


31247/31247 [==============================] - 855s 27ms/step - loss: 1.3105 - accuracy: 0.5919 - val_loss: 1.5859 - val_accuracy: 0.5369
Epoch 3/10
17638/31247 [===============>..............] - ETA: 6:13 - loss: 1.2874 - accuracy: 0.5971

KeyboardInterrupt: 